In [1]:

import uesgraphs as ug

from uesgraphs.examples import e1_example_readme as e1

import os
import sys
from datetime import datetime
import tempfile

In [2]:
import uesgraphs.analysis as analysis


In [3]:

workspace = e1.workspace_example("e14")

dir_ues = os.path.dirname(os.path.dirname(workspace))
pinola_json = os.path.join(dir_ues, "workspace", "e11", "inputs","test_modelgen", "Pinola", "nodes.json")
pinola_sim_data = os.path.join(dir_ues,"uesgraphs","data","Pinola_low_temp_network_inputs.mat")
pinola_sim_data = r"E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip"
pinola_json =r"E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\transurban_seestadt_uesgraphs.json"
sysm_model = r"E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\model.json"
if not os.path.exists(pinola_json):
    raise FileNotFoundError(f"File {pinola_json} not found."
                            "Please run example e11 to generate network topology.")


In [4]:

graph = ug.UESGraph()
graph.from_json(path = pinola_json, network_type="heating")
graph.graph["name"] = "pinola"
graph.graph["supply_type"] = "supply"

start_date=datetime(2024, 1, 1) 
end_date=datetime(2024, 1, 7)
    


read nodes...
******
 input_ids were {'buildings': None, 'nodes': 'f607cb63-8aed-446c-9970-9cd8380434f8', 'pipes': None, 'supplies': None}
...finished


In [5]:
#gr = analysis.assign_simulation_data(graph, pinola_sim_data, start_date, end_date,aixlib_version="2.0.0",auto_retry=True)

In [6]:
from uesgraphs.systemmodels import utilities as ut
sysm_graph = ut.load_system_model_from_json(sysm_model)

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\3\SystemModelHeating_20250828_184729.log
Model loaded from E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\model.json


In [7]:
from uesgraphs.systemmodels import utilities as ut
sysm_graph = ut.load_system_model_from_json(sysm_model)
from uesgraphs.analysis.data_handling import graph_transformation
port_mapping = graph_transformation.map_system_model_to_uesgraph(sysm_graph,graph)

In [8]:
# 1. Supply Type und Masks laden
supply_type = graph.graph["supply_type"]  # "supply" oder "return"
supply_type_prefix = {"supply": "", "return": "R"}

from uesgraphs.analysis.data_handling.data_handling import get_MASKS
masks = get_MASKS("2.1.0")  # oder deine gewünschte Version

# 2. Filter-Liste bauen (wie im existierenden Code)
filter_list = []
for edge in graph.edges:
    pipe_code = graph.edges[edge]["name"]
    for mask_type, mask_pattern in masks.items():
        var_name = mask_pattern.format(
            pipe_code=pipe_code, 
            type=supply_type_prefix[supply_type]
        )
        filter_list.append(var_name)

print(f"Filter-Liste mit {len(filter_list)} Variablen:")
for var in filter_list[:10]:  # Erste 10 zeigen
    print(f"  - {var}")

# 3. Daten laden
from uesgraphs.analysis.data_handling.data_handling import process_simulation_result
df = process_simulation_result(file_path=pinola_sim_data, filter_list=filter_list)

print(f"\nDataFrame Shape: {df.shape}")
print(f"Spalten: {list(df.columns)}")

uesgraphs.analysis.data_handling.data_handling.get_MASKS - INFO - Using AixLib version 2.1.0 masks
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Found existing gzip file: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Validating 72 required columns in: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip


Filter-Liste mit 72 Variablen:
  - networkModel.pipe10021010.port_a.m_flow
  - networkModel.pipe10021010.dp
  - networkModel.pipe10021010.port_a.p
  - networkModel.pipe10021010.port_b.p
  - networkModel.pipe10021010.sta_a.T
  - networkModel.pipe10021010.sta_b.T
  - networkModel.pipe10021014.port_a.m_flow
  - networkModel.pipe10021014.dp
  - networkModel.pipe10021014.port_a.p
  - networkModel.pipe10021014.port_b.p


uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - All required columns found in data file
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Starting parquet file processing: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Successfully loaded 35041 rows, 72 columns



DataFrame Shape: (35041, 72)
Spalten: ['networkModel.pipe10021010.port_a.m_flow', 'networkModel.pipe10021010.dp', 'networkModel.pipe10021010.port_a.p', 'networkModel.pipe10021010.port_b.p', 'networkModel.pipe10021010.sta_a.T', 'networkModel.pipe10021010.sta_b.T', 'networkModel.pipe10021014.port_a.m_flow', 'networkModel.pipe10021014.dp', 'networkModel.pipe10021014.port_a.p', 'networkModel.pipe10021014.port_b.p', 'networkModel.pipe10021014.sta_a.T', 'networkModel.pipe10021014.sta_b.T', 'networkModel.pipe10021025.port_a.m_flow', 'networkModel.pipe10021025.dp', 'networkModel.pipe10021025.port_a.p', 'networkModel.pipe10021025.port_b.p', 'networkModel.pipe10021025.sta_a.T', 'networkModel.pipe10021025.sta_b.T', 'networkModel.pipe10041006.port_a.m_flow', 'networkModel.pipe10041006.dp', 'networkModel.pipe10041006.port_a.p', 'networkModel.pipe10041006.port_b.p', 'networkModel.pipe10041006.sta_a.T', 'networkModel.pipe10041006.sta_b.T', 'networkModel.pipe10041007.port_a.m_flow', 'networkModel.pip

In [9]:
i = 1001
node_ports = port_mapping[i]  # Liste aller Ports für diesen Node
collected_values = {"T": [], "p": []}

# Mapping für flexible Attributnamen im Graph
attribute_names = {"T": "temperature", "p": "press_flow"}  # Konfigurierbar!

for port in node_ports:  # Iteriere über ALLE Ports des Nodes
    port_splitted = port.split(".")
    if port_splitted[1].endswith("a"):
        #port is an a port
        temp = f"networkModel.{port_splitted[0]}.sta_a.T"
        press = f"networkModel.{port_splitted[0]}.port_a.p"
        t_value = df[temp].iloc[0]
        p_value = df[press].iloc[0]
        collected_values["T"].append(t_value)
        collected_values["p"].append(p_value)

# Validiere und weise zu
for size in ["T", "p"]:
    unique_vals = set(collected_values[size])
    if len(unique_vals) != 1:
        raise ValueError(f"Node {i}: Inkonsistente {size}-Werte gefunden: {collected_values[size]}")
    graph.nodes[i][attribute_names[size]] = collected_values[size][0]

In [10]:
print(f"Port Mapping für Knoten {i}: {port}")
pipe_mask = df.columns.str.contains('pipe10021014.')
df_pipe = df.loc[:, pipe_mask]
print(f"Gefundene Spalten: {df.columns[pipe_mask].tolist()}")

Port Mapping für Knoten 1001: pipe10021025.port_a
Gefundene Spalten: ['networkModel.pipe10021014.port_a.m_flow', 'networkModel.pipe10021014.dp', 'networkModel.pipe10021014.port_a.p', 'networkModel.pipe10021014.port_b.p', 'networkModel.pipe10021014.sta_a.T', 'networkModel.pipe10021014.sta_b.T']


In [11]:
edges = list(graph.edges)
for edge in edges:
    print(f"Edge {edge} has data: {graph.edges[edge]}")

Edge (1001, 1005) has data: {'diameter': 0.0697, 'length': 62.67448664094058, 'pipeID': '10021010', 'name': '10021010', 'node_0': 1002, 'node_1': 1010}
Edge (1001, 1006) has data: {'diameter': 0.2101, 'length': 40.10060652984215, 'pipeID': '10021014', 'name': '10021014', 'node_0': 1002, 'node_1': 1014}
Edge (1001, 1009) has data: {'diameter': 0.2101, 'length': 23.793988913832845, 'pipeID': '10021025', 'name': '10021025', 'node_0': 1002, 'node_1': 'supply1'}
Edge (1002, 1003) has data: {'diameter': 0.1325, 'length': 23.15227945754485, 'pipeID': '10041006', 'name': '10041006', 'node_0': 1004, 'node_1': 1006}
Edge (1002, 1004) has data: {'diameter': 0.1325, 'length': 67.21343661070965, 'pipeID': '10041007', 'name': '10041007', 'node_0': 1004, 'node_1': 1007}
Edge (1002, 1011) has data: {'diameter': 0.0539, 'length': 25.20026053450003, 'pipeID': '10041027', 'name': '10041027', 'node_0': 1004, 'node_1': 'w11_2'}
Edge (1003, 1008) has data: {'diameter': 0.2101, 'length': 43.10762795752977, '

In [12]:
    graph = analyze.assign_data_to_uesgraphs(graph,sim_data = pinola_sim_data,
                                             start_date=start_date,
                                             end_date=end_date,
                                             aixlib_version="2.1.0") #aixlib version is needed to assign data properly
    
    ### Plotting
    #Visuals cant handle series data, so we just take the mean values, but single time points are possible
    vis = ug.Visuals(graph)
    for edge in graph.edges:
        graph.edges[edge]["m_flow_mean"] = graph.edges[edge]["m_flow"].mean()
    vis.show_network(show_plot=False,
                           scaling_factor=1,
                           scaling_factor_diameter=50,
                           label_size=15,
                           ylabel="Mean mass flow [kg/s]",
                           generic_extensive_size="m_flow_mean",
                           save_as=os.path.join(workspace, "m_flow.png"),
                           timestamp=f"{graph.graph["name"]}: Mean mass flow"
                           )
    
    for node in graph.nodes:
        graph.nodes[node]["press_flow_mean"] = graph.nodes[node]["press_flow"].mean()
    vis.show_network(show_plot=False,
                           scaling_factor=1,
                           scaling_factor_diameter=50,
                           ylabel="Mean pressure [Pa]",
                           label_size=15,
                           generic_intensive_size="press_flow_mean",
                           save_as=os.path.join(workspace, "press_flow.png"),
                           timestamp=f"{graph.graph["name"]}: Mean pressure flow"
                           )
    
    df = analyze.pump_power_analysis(graph, True, r"D:\rka-lko\work\2025_04_analysis\3")

if __name__ == "__main__":
    main()

IndentationError: unexpected indent (2453415739.py, line 1)

# New flow

In [27]:
AIXLIB_MASKS = {
    "2.1.0": {
        "edge": {
            # Extensive properties - same value at both ports
            "m_flow": "networkModel.pipe{pipe_code}{type}.port_a.m_flow",
            "dp": "networkModel.pipe{pipe_code}{type}.dp",
        },
        "node": {
            # Intensive properties - may differ between ports
            "pressure": {
                "port_a": "networkModel.pipe{pipe_code}{type}.port_a.p",
                "port_b": "networkModel.pipe{pipe_code}{type}.port_b.p"
            },
            "temperature": {
                "port_a": "networkModel.pipe{pipe_code}{type}.sta_a.T", 
                "port_b": "networkModel.pipe{pipe_code}{type}.sta_b.T"
            }
        }
    },
    "2.0.0": {
        "edge": {
            # Extensive properties - same value at both ports
            "m_flow": "networkModel.pipe{pipe_code}{type}.port_a.m_flow",
        },
        "node": {
            # Intensive properties - may differ between ports
            "pressure": {
                "port_a": "networkModel.pipe{pipe_code}{type}.port_a.p",
                "port_b": "networkModel.pipe{pipe_code}{type}.ports_b[1].p"
            },
            "temperature": {
                "port_a": "networkModel.pipe{pipe_code}{type}.sta_a.T",
                "port_b": "networkModel.pipe{pipe_code}{type}.sta_b[1].T"
            }
        }
    }
}

In [28]:
def get_supply_type_prefix(graph):
    supply_type = graph.graph.get("supply_type", "supply")
    supply_type_prefix = {"supply": "", "return": "R"}
    return supply_type_prefix.get(supply_type, "")

def build_filter_list_pipe(graph,MASK, logger=None):
    if logger is None:
        import logging
        logger = logging.getLogger(__name__)
    mask_sim = []
    for category_name, category in MASK.items():
        if category_name in ["edge", "node"]:
            if category_name == "edge":
                # Edge values: direct access to patterns
                mask_sim.extend(category.values())
            else:  # node
                # Node values: nested structure - extract all port patterns
                for attribute_patterns in category.values():
                    mask_sim.extend(attribute_patterns.values())

    type_prefix = get_supply_type_prefix(graph)

    filter_list = []
    for edge in graph.edges:
        pipe_code = graph.edges[edge]["name"]
        for entry_sim in mask_sim:
            var_name = entry_sim.format(
                pipe_code=pipe_code, 
                type=type_prefix
            )
            filter_list.append(var_name)
    return filter_list



In [29]:

def assign_node_values(graph, df, port_mapping, mask, time_index=0, logger=None):
    """
    Assigns node values from simulation data using flexible mask configuration.
    
    Processes intensive properties (pressure, temperature) that may differ between 
    ports of the same pipe, unlike extensive properties (mass flow) that are 
    identical at both ports.
    
    Args:
        graph: NetworkX graph with nodes to assign values to
        df: DataFrame containing simulation data
        port_mapping: Dict mapping node_ids to list of connected ports.
                     Example: {1: ['pipe001.port_a', 'pipe002.port_b'], 
                              2: ['pipe001.port_b', 'pipe003.port_a']}
                     Source: [Method name if known, otherwise leave empty]
        mask: Mask dictionary containing node configuration for intensive properties
        time_index: Time step index to extract from df (default: 0)
        logger: Logger instance (optional, creates terminal logger if None)
    """
    if logger is None:
        logger = set_up_terminal_logger(f"{__name__}.assign_node_values")
    
    # Extract node configuration
    node_config = mask.get("node", {})
    if not node_config:
        logger.error("No 'node' configuration found in mask")
        return
    
    type_suffix = get_supply_type_prefix(graph)
    
    stats =  {
        'processed_count': 0,
        'inconsistency_count': 0, 
        'pattern_conflicts': 0
    }
    
    for node_id, node_ports in port_mapping.items():
        if not node_ports:
            continue
            
        _assign_attributes_to_node(
            graph, node_id, node_ports, df, node_config,
            type_suffix, stats, logger
        )
        
        stats['processed_count'] += 1
    
    logger.info(f"Assignment completed:")
    logger.info(f"  Processed nodes: {stats['processed_count']}")
    logger.info(f"  Within-pattern inconsistencies: {stats['inconsistency_count']}")
    logger.info(f"  Cross-pattern conflicts: {stats['pattern_conflicts']}")

def _assign_attributes_to_node(graph, node_id, node_ports, df, config, 
                              type_suffix, stats, logger):
    """Assign all attributes to a single node.
        config: {"attribute": {"port_suffix": "pattern_with_{pipe_code}"}}
                ex.: {"temperature": {"port_a": "networkModel.pipe{pipe_code}{type}.sta_a.T"}}
    """
    
    for attribute_name, port_patterns in config.items():
        """Collect values for a specific attribute from all relevant ports.
        attribute_name: e.g. "temperature"
        port_patterns: e.g. {"port_a": "networkModel.pipe{pipe_code}{type}.sta_a.T",
                        "port_b": "networkModel.pipe{pipe_code}{type}.sta_b.T"}
        """
        series_list = []
        for port in node_ports:
            
            pipe_name, port_suffix = _parse_port_identifier(port)
            if port_suffix in port_patterns:
                pattern = port_patterns[port_suffix]
                column_name = pattern.format(pipe_code=pipe_name, type=type_suffix)
            
                if column_name in df.columns:
                    series = df[column_name]
                    series_list.append(series)
                else:
                    logger.debug(f"Column not found: {column_name}")

        if len(series_list) == 0:
            logger.debug(f"No data found for {attribute_name} at node {node_id}")
            continue

        # Check if all series are identical
        if len(series_list) > 1:
            all_equal = all(series_list[0].equals(series) for series in series_list[1:])
            if not all_equal:
                logger.warning(f"Node {node_id}: Inconsistent {attribute_name} time series found")
                stats['inconsistency_count'] += 1
        
        # Use first series as result
        graph.nodes[node_id][attribute_name] = series_list[0]        

def _parse_port_identifier(port):
    """Parse port identifier to extract pipe name and port suffix."""
    port_parts = port.split(".")
    if len(port_parts) < 2:
        raise ValueError(f"Invalid port format: {port}. Expected 'pipe_name.port_suffix'")
    return port_parts[0].replace("pipe", ""), port_parts[1]


## Edges
def assign_edge_data(graph, MASK, df):
    type_suffix = get_supply_type_prefix(graph)
    for edge in graph.edges:
        for edge_variable, variable_mask in MASK["edge"].items():
            pipe_name = graph.edges[edge]["name"]
            variable_name = variable_mask.format(pipe_code=pipe_name, type=type_suffix)
            graph.edges[edge][edge_variable] = df[variable_name]
        
## validation
def validate_graph_attributes(graph, mask, reference_df, logger=None):
    """
    Validates graph edge and node attributes against a reference DataFrame.
    
    Checks if the length of attribute arrays in edges and nodes matches the 
    number of rows in the reference DataFrame.
    
    Args:
        graph: NetworkX Graph object with edges and nodes
        mask (dict): Dictionary with "edge" and "node" keys containing 
                    attributes to validate
        reference_df (pd.DataFrame): Reference DataFrame for length comparison
        logger (logging.Logger, optional): Logger instance. If None, a 
                                         terminal logger will be created.
    
    Returns:
        bool: True if all validations pass, False otherwise
        
    Raises:
        ValueError: For critical validation errors
    """
    if logger is None:
        logger = set_up_terminal_logger(f"{__name__}.validate_graph_attributes")
    
    logger.info("Starting graph attribute validation...")
    
    expected_length = reference_df.shape[0]
    validation_passed = True
    errors = []
    
    # Validate edge attributes
    logger.info(f"Validating edge attributes for {len(graph.edges)} edges...")
    
    if "edge" in mask:
        for edge_idx, edge in enumerate(graph.edges):
            for edge_attr in mask["edge"].keys():
                if edge_attr in graph.edges[edge]:
                    actual_length = len(graph.edges[edge][edge_attr])
                    
                    if actual_length != expected_length:
                        error_msg = (
                            f"Edge {edge} - Attribute '{edge_attr}': "
                            f"length {actual_length} != expected length {expected_length}"
                        )
                        logger.error(error_msg)
                        errors.append(error_msg)
                        validation_passed = False
                    else:
                        logger.debug(
                            f"Edge {edge} - Attribute '{edge_attr}': OK "
                            f"(length: {actual_length})"
                        )
                else:
                    warning_msg = f"Edge {edge} - Attribute '{edge_attr}' not found"
                    logger.warning(warning_msg)
    else:
        logger.warning("No edge attributes defined in MASK")
    
    # Validate node attributes
    logger.info(f"Validating node attributes for {len(graph.nodes)} nodes...")
    
    if "node" in mask:
        for node_idx, node in enumerate(graph.nodes):
            for node_attr in mask["node"].keys():
                if node_attr in graph.nodes[node]:
                    actual_length = len(graph.nodes[node][node_attr])
                    
                    if actual_length != expected_length:
                        error_msg = (
                            f"Node {node} - Attribute '{node_attr}': "
                            f"length {actual_length} != expected length {expected_length}"
                        )
                        logger.error(error_msg)
                        errors.append(error_msg)
                        validation_passed = False
                    else:
                        logger.debug(
                            f"Node {node} - Attribute '{node_attr}': OK "
                            f"(length: {actual_length})"
                        )
                else:
                    warning_msg = f"Node {node} - Attribute '{node_attr}' not found"
                    logger.warning(warning_msg)
    else:
        logger.warning("No node attributes defined in MASK")
    
    # Validation summary
    if validation_passed:
        logger.info("✅ Graph attribute validation completed successfully")
    else:
        logger.error(f"❌ Graph attribute validation failed: {len(errors)} errors")
        for error in errors[:5]:  # Show maximum 5 errors in summary
            logger.error(f"  - {error}")
        if len(errors) > 5:
            logger.error(f"  ... and {len(errors) - 5} more errors")
    
    return validation_passed


In [31]:
from uesgraphs.analysis.data_handling.data_handling import validate_columns_exist, check_supply_type, set_up_file_logger, prepare_DataFrame, set_up_terminal_logger
aixlib_version = "2.1.0"
time_interval = "15min"

from uesgraphs.systemmodels import utilities as ut
sysm_graph = ut.load_system_model_from_json(sysm_model)
from uesgraphs.analysis.data_handling import graph_transformation
port_mapping = graph_transformation.map_system_model_to_uesgraph(sysm_graph,graph)

logger = set_up_file_logger("all")
MASK = AIXLIB_MASKS[aixlib_version]
filter_list = build_filter_list_pipe(graph,MASK=MASK,logger=logger)
m = validate_columns_exist(file_path=pinola_sim_data, required_columns=filter_list,logger=logger)

df = process_simulation_result(file_path=pinola_sim_data, filter_list=filter_list, logger=logger)
df = prepare_DataFrame(df, start_date=start_date, end_date=end_date, 
                             time_interval=time_interval, logger=logger)

assign_node_values(graph, df, port_mapping, MASK, logger=logger)
assign_edge_data(graph, MASK, df)

validate_graph_attributes(graph, MASK, df,logger=logger)

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\3\all_20250828_185501.log


True

__main__.validate_graph_attributes - INFO - Starte Graph-Attribut-Validierung...
__main__.validate_graph_attributes - INFO - Validiere Edge-Attribute für 12 Edges...
__main__.validate_graph_attributes - INFO - Validiere Node-Attribute für 13 Nodes...
__main__.validate_graph_attributes - INFO - ✅ Graph-Attribut-Validierung erfolgreich abgeschlossen


True